In [1]:
from py2neo import Graph

ip = "localhost"
graph = Graph("bolt://%s:7687"%ip, password="123456")

In [2]:
from sqlalchemy import create_engine
import pandas as pd

username = "shaw"
password = "1234"
port = 3306
database = "community"

engine = create_engine('mysql+mysqldb://%s:%s@localhost:%i/%s'
                       %(username, password, port, database))

pd.read_sql_query("show tables", engine)

,Tables_in_community
0,History
1,Person
2,component_sz
3,valid_community


```SQL
DROP TABLE  `community`.`History`;

CREATE TABLE `community`.`History` ( `id` INT NOT NULL AUTO_INCREMENT , `name` VARCHAR(10) NOT NULL , `community` INT NULL , `core` BOOLEAN NOT NULL , `date` INT NOT NULL , PRIMARY KEY (`id`)) ENGINE = InnoDB;
```

In [3]:
# sql = """
# DROP TABLE  `community`.`History`;

# CREATE TABLE `community`.`History` ( `id` INT NOT NULL AUTO_INCREMENT , `name` VARCHAR(10) NOT NULL , `community` INT NULL , `core` BOOLEAN NOT NULL , `date` INT NOT NULL , PRIMARY KEY (`id`)) ENGINE = InnoDB;
# """
# engine.execute(sql)

In [4]:
cql = """
CALL algo.unionFind.stream("Person", "KNOWS")
YIELD nodeId, setId
WITH algo.getNodeById(nodeId).name as name, setId as component
MATCH (p)
WHERE p.name = name
SET p += {component: component}
RETURN p;
"""

df = graph.run(cql).to_data_frame()
df.head()

,p
0,"{'name': 'Delilah', 'component': 1}"
1,"{'name': 'Pax', 'component': 1}"
2,"{'name': 'Bennett', 'component': 1}"
3,"{'name': 'Tilda', 'component': 1}"
4,"{'name': 'Dari', 'component': 1}"


In [5]:
cql = """
MATCH (p)--(r)
WITH p.name AS name, count(r) AS degree
MATCH (p)
where p.name = name
SET p += {degree: degree}
RETURN p;
"""

df = graph.run(cql).to_data_frame()
df.head()

,p
0,"{'degree': 3, 'name': 'Mackenzie', 'component'..."
1,"{'degree': 4, 'name': 'Dari', 'component': 1}"
2,"{'degree': 5, 'name': 'Sam', 'component': 1}"
3,"{'degree': 5, 'name': 'Pip', 'component': 1}"
4,"{'degree': 3, 'name': 'Mattie', 'component': 1}"


#### Inherit

In [6]:
cql = """
MATCH (p) SET p += {was_core: p.core};
"""

graph.run(cql)

In [7]:
cql = """
MATCH (p) SET p += {core: False};
"""

graph.run(cql)

In [8]:
cql = """
MATCH (p) 
WHERE p.was_core = FALSE
REMOVE p.community;
"""

graph.run(cql)

#### Merge & Propagate

In [9]:
cql = """
MATCH (p) 
WHERE p.was_core = True 
WITH p.component AS component, MAX(p.community) AS community 
MATCH (p) 
WHERE p.component = component 
SET p += {community: community};
"""

graph.run(cql)

#### Assign

In [10]:
cql = """
MATCH (p)
WITH p.component as c, MAX(p.degree) as maxdegree
WHERE maxdegree > 1
WITH c, maxdegree
MATCH (p)
WHERE p.degree = maxdegree
  and p.component = c
WITH MAX(p.name) as name, p.degree as degree, p.component as component
MATCH (p)
WHERE p.name = name
SET p:Core, p += {core: True}
RETURN p;
"""

graph.run(cql)

In [11]:

query = """
MATCH (n)
RETURN 
  n.name as name,
  n.component as component,
  n.core as core,
  n.community as community,
  CASE WHEN n.degree IS NULL THEN 0 else n.degree END AS score
"""

df = graph.run(query).to_data_frame()
df.to_sql('Person', engine, if_exists='replace')
df.head()

,community,component,core,name,score
0,None,1,False,Delilah,4
1,None,1,False,Pax,2
2,None,1,False,Bennett,3
3,None,1,False,Tilda,3
4,None,1,False,Dari,4


#### Detect

In [12]:
sql = """
drop table if exists component_sz;
create table component_sz as
select 
  component,
  count(*) as sz
from Person
where community is null
group by component
order by sz desc;
"""

engine.execute(sql)
pd.read_sql_query("select * from component_sz", engine)

,component,sz
0,37,39
1,1,24
2,19,15
3,26,14
4,17,4
5,11,1
6,30,1
7,40,1
8,46,1


In [13]:
sql = """
drop table if exists valid_community;
create table valid_community as
select 
  a.component,
  a.sz,
  count(*) + (select coalesce(max(community), 0) from History) as community
from component_sz a
join component_sz b
on a.sz <= b.sz
where a.sz >= 3
group by component, sz
order by community;
"""

engine.execute(sql)
pd.read_sql_query("select * from valid_community", engine).head()

,component,sz,community
0,37,39,1
1,1,24,2
2,19,15,3
3,26,14,4
4,17,4,5


In [14]:
sql = """
insert into History (core, name, community, date)
select 
  a.core as core
  ,a.name as name
  ,b.community as community
  ,(select coalesce(max(date), 0) from History) + 1 as date
from Person as a
left join valid_community as b
on a.component = b.component;
"""

engine.execute(sql)
pd.read_sql_query("select * from History where date = (select coalesce(max(date), 0) from History)", engine).head()

,id,name,community,core,date
0,1,Asher,1.0,0,1
1,2,Kyle,1.0,0,1
2,3,Vesper,1.0,0,1
3,4,Jude,1.0,0,1
4,5,Pippa,1.0,0,1


In [15]:
df = pd.read_sql_query("select * from History where date = (select coalesce(max(date), 0) from History) and community is not null", engine)
df.head()

,id,name,community,core,date
0,1,Asher,1,0,1
1,2,Kyle,1,0,1
2,3,Vesper,1,0,1
3,4,Jude,1,0,1
4,5,Pippa,1,0,1


In [16]:
cql_tmp = """
MATCH (p)
WHERE p.name = \"%s\"
SET p += {community: %i};
"""

In [17]:
for i, row in df.iterrows():
    graph.run(cql_tmp%(row["name"], row["community"]))